In [ ]:

# IAM/RBAC Optimization (Scaled)
# ------------------------------
# 20 Users, 10 Roles, 30 Permissions

# !pip install pulp pandas

import pulp
import random
import pandas as pd

random.seed(42)

num_users = 20
num_roles = 10
num_permissions = 30

users = [f'u{i}' for i in range(num_users)]
roles = [f'r{j}' for j in range(num_roles)]
permissions = [f'p{k}' for k in range(num_permissions)]

required_permissions = {
    u: random.sample(permissions, random.randint(5, 10)) for u in users
}

sod_conflicts = [('r0', 'r1'), ('r2', 'r3'), ('r4', 'r5'), ('r6', 'r7')]

max_roles_per_user = 3
max_users_per_role = 6

model = pulp.LpProblem("IAM_RBAC_Scaled_Optimization", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", [(u, r) for u in users for r in roles], cat='Binary')
y = pulp.LpVariable.dicts("y", [(r, p) for r in roles for p in permissions], cat='Binary')
z = pulp.LpVariable.dicts("z", [(u, p) for u in users for p in permissions], cat='Binary')

model += pulp.lpSum(
    z[(u, p)] for u in users for p in permissions if p not in required_permissions[u]
)

for u in users:
    for p in permissions:
        model += z[(u, p)] <= pulp.lpSum([x[(u, r)] for r in roles])
        model += z[(u, p)] <= pulp.lpSum([y[(r, p)] for r in roles])
        for r in roles:
            model += z[(u, p)] >= x[(u, r)] + y[(r, p)] - 1

for u in users:
    for (r1, r2) in sod_conflicts:
        model += x[(u, r1)] + x[(u, r2)] <= 1

for u in users:
    model += pulp.lpSum([x[(u, r)] for r in roles]) <= max_roles_per_user

for r in roles:
    model += pulp.lpSum([x[(u, r)] for u in users]) <= max_users_per_role

for u in users:
    req_set = required_permissions[u]
    model += pulp.lpSum([z[(u, p)] for p in req_set]) >= min(3, len(req_set))

model.solve(pulp.PULP_CBC_CMD(msg=False))

assignments = []
lp_violations = 0
sod_violations = 0

for u in users:
    assigned_roles = [r for r in roles if pulp.value(x[(u, r)]) == 1]
    assigned_perms = [p for p in permissions if pulp.value(z[(u, p)]) == 1]
    req = required_permissions[u]
    lp_violations += len([p for p in assigned_perms if p not in req])
    for (r1, r2) in sod_conflicts:
        if r1 in assigned_roles and r2 in assigned_roles:
            sod_violations += 1
    assignments.append({
        "User": u,
        "Assigned Roles": assigned_roles,
        "Assigned Permissions": assigned_perms,
        "Required Permissions": req
    })

df = pd.DataFrame(assignments)
print("=== Optimized IAM Assignments ===")
print(df)

total_perm_assigned = sum(len(a["Assigned Permissions"]) for a in assignments)
lpvr = lp_violations / total_perm_assigned if total_perm_assigned else 0
roles_used = sum(any(pulp.value(x[(u, r)]) == 1 for u in users) for r in roles)

print("\n=== Evaluation Metrics ===")
print(f"Least Privilege Violation Rate (LPVR): {lpvr:.2%}")
print(f"Separation of Duty (SoD) Violations: {sod_violations}")
print(f"Total Roles Used: {roles_used}")
